In [1]:
using Knet
using AutoGrad
using Images
using ImageMagick
using Random
using Statistics
import Base: length, size, iterate, eltype, IteratorSize, IteratorEltype, haslength, @propagate_inbounds, repeat, rand, tail
import .Iterators: cycle, Cycle, take


Start by loading list of images to be used

In [3]:
dataPath = "GeNeVA-v1\\i-CLEVR\\images"
files = readdir(dataPath)

60023-element Array{String,1}:
 ".ipynb_checkpoints"
 "CLEVR_test_000000.png"
 "CLEVR_test_000000_0.png"
 "CLEVR_test_000000_1.png"
 "CLEVR_test_000000_2.png"
 "CLEVR_test_000000_3.png"
 "CLEVR_test_000000_4.png"
 "CLEVR_test_000001.png"
 "CLEVR_test_000001_0.png"
 "CLEVR_test_000001_1.png"
 "CLEVR_test_000001_2.png"
 "CLEVR_test_000001_3.png"
 "CLEVR_test_000001_4.png"
 ⋮
 "CLEVR_val_001998.png"
 "CLEVR_val_001998_0.png"
 "CLEVR_val_001998_1.png"
 "CLEVR_val_001998_2.png"
 "CLEVR_val_001998_3.png"
 "CLEVR_val_001998_4.png"
 "CLEVR_val_001999.png"
 "CLEVR_val_001999_0.png"
 "CLEVR_val_001999_1.png"
 "CLEVR_val_001999_2.png"
 "CLEVR_val_001999_3.png"
 "CLEVR_val_001999_4.png"

Select images containing single object

In [4]:
singleObj = files[endswith.(files, "_0.png")]
singleObj = ["$(dataPath)\\$(file)" for file in singleObj]

10004-element Array{String,1}:
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000000_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000001_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000002_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000003_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000004_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000005_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000006_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000007_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000008_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000009_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000010_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000011_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000012_0.png"
 ⋮
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001988_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001989_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001990_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001991_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR

Create iterator for Clevr Dataset

In [5]:
struct ClevrData
    files
    batchsize::Int
    #shuffle::Bool
    num_instances::Int

    function ClevrData(files; batchsize::Int=64) #shuffle::Bool=false)
        new(files, batchsize, length(files))
    end
end

In [6]:
function length(d::ClevrData)
    nFullBatches, rem = divrem(d.num_instances, d.batchsize)
    nFullBatches + (rem > 0)*1
end

length (generic function with 204 methods)

In [7]:
function load_image(filename)
    img = load(filename)
    img = imresize(img, (64,64))
    img = Float64.(channelview(img))
    img = img[1:3,:,:]
end

function load_batch(batches, i)
    #batch = [load_image(image) for image in batches[i,:]]
    batch = load_image.(batches[i,:])
end

load_batch (generic function with 1 method)

In [14]:
function iterate(d::ClevrData, state=collect(1:d.num_instances))
    if length(state) > 0
        batch = d.files[state[1:(length(state) < d.batchsize ? end : d.batchsize)]]
        state  = state[d.batchsize+1:end]
        return (load_image.(batch)), state
    end
end

iterate (generic function with 422 methods)

In [17]:
clevrDataset = ClevrData(singleObj)

ClevrData(["GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000000_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000001_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000002_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000003_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000004_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000005_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000006_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000007_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000008_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000009_0.png"  …  "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001990_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001991_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001992_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001993_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001994_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001995_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001996_0.png", "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001997_0.png", "GeN

In [18]:
for batch in clevrDataset
    println(length(batch))
end

128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
128
20


In [ ]:
function gLoss(G::Generator, D::Discriminator, z)
    yPr = D(G(z))
    return -0.5*mean(log.(yPr))
end

In [ ]:
function dLoss(D::Discriminator, realIms, fakeIms)
    realLoss = -mean(log.(D(realIms)))
    fakeLoss = -mean(log.(1 .- D(fakeIms)))
    return 0.5*(realLoss+fakeLoss)
end

In [8]:
# function make_minibatch(names, batchSize = 128)
#     rem = mod(length(names), batchSize)
#     names = reshape(names[1:end-rem], :, batchSize)
# end

# batches = make_minibatch(singleObj, 64)
# batch = load_batch(batches, 1)

156×64 Array{String,2}:
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000000_0.png"  …  "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001824_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000001_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001825_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000002_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001826_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000003_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001827_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000004_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001828_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000005_0.png"  …  "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001829_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000006_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001830_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000007_0.png"     "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_val_001831_0.png"
 "GeNeVA-v1\\i-CLEVR\\images\\CLEVR_test_000008_0.png"     "GeNeVA-v1\\i-CLEVR\\